# Parsing Show Command Output with TextFSM Python Module

Given a text file with show command output, parse the output using a TextFSM template.  This will get all the show command data into a Python structure that can be used to make decisions.
Remember to install textfsm into your virtual environment

`pip install textfsm`

In [1]:
import textfsm
import os

In [ ]:
pwd

In [2]:
ls

TextFSM_Sandbox.ipynb
TextFSM_Sandbox.py
arista_eos_show_ip_interface_brief.textfsm
arista_sw01_show_ip_int_br_output.txt
python_script_template.py


### Safely Opening a File is something that you will need to do alot!
Lets see what it takes to create a reusable function we could use for this

In [3]:
def open_file(file_path):
    """
    Given a file path, this function verifies that the file path is valid
    and points to a file and then attempts to open the file and read
    in its contents as a string.
    
    file_path: File path to file.
    
    return: file_data_string: Function will retur the contents of the file
    in a string if sucessful, otherwise it will return an empty string.
    
    """
    print(f"Attempting to open file {file_path}")
    if os.path.isfile(file_path):
        # TODO: Put this in a try/except block to really error proof 
        with open(file_path) as f:
            file_data_string = f.read()
    else:
        print(f"\nERROR! File path provided is invalid. Verify that the file exists at the path provided.\nFailed to open {file_path}")
        file_data_string = ""
          
    return file_data_string

In [4]:
with open("TestFailure.txt") as f:
    file_data_string = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'TestFailure.txt'

In [5]:
open_file("TestFailure.txt")

Attempting to open file TestFailure.txt

ERROR! File path provided is invalid. Verify that the file exists at the path provided.
Failed to open TestFailure.txt


''

In [6]:
my_data = open_file("arista_sw01_show_ip_int_br_output.txt")

Attempting to open file arista_sw01_show_ip_int_br_output.txt


In [7]:
my_data

'sw02#show ip int br\n                                                                         Address\nInterface       IP Address            Status     Protocol         MTU    Owner\n--------------- --------------------- ---------- ------------- --------- -------\nEthernet1       192.168.56.107/24     up         up              1500\nEthernet2       192.168.194.10/24     up         up              1500\nEthernet3       192.168.62.10/24      up         up              1500\nManagement1     10.1.10.55/24         up         up              1500\n\nsw02#\nsw02#\n\n'

### Parsing show command output with TextFMS is something else that will happen often
Lets see what it takes to create another reusable function we could use for this

In [9]:
def textfsm_parser(template_fp, data_to_parse):

    """
    This function uses the provided template file path, template_fp, to parse the provided string of data, 
    data_to_parse.
    
    template_fp: The full file path to the template file or a template file in the local directory with this script.
    
    data_to_parse: String of data to parse. Typically the output of the show command which corresponds to the 
    TextFMS template being used.
    
    returns: fsm: The resulting parsed TextFSM object.
    
    Tip:  
    fsm.header provides the column headers for each "column" of data (positional element in the list)
    fsm._results provides the resulting list (same value as the variable result below)
    fsm.values provides the template values to parse out of the data (the regex for each column header)
    
    """
    
    if os.path.isfile(template_fp):
        print(f"Using TextFSM Template {template_fp}")
        with open(template_fp) as template_fh:
            # TextFSM method needs a file handle which is why we are not using our open_file function
            fsm = textfsm.TextFSM(template_fh)
            result = fsm.ParseText(my_data)
    else:
        print(f"ERROR! {template_fp} is not a valid file or not in the given path!")
        fsm = ""

    print(f"FSM Object fsm (basically the parsing template to use):\n{fsm}\n")
    print(f"FSM Object fsm.header:\n{fsm.header}\n")
    print(f"FSM Object fsm.values:\n{fsm.values}\n")
    print(f"FSM Object fsm._result:\n{fsm._result}\n")
    print(f"The contents of the result variable (same as fsm._result): \n{result}\n")
    print(f"FSM Object fsm.value_map:\n{fsm.value_map}\n")
    print("\ndir of fsm dir(fsm) to look at available methods from the TextFSM Object.")
    print(dir(fsm))
    
    # Returning the fsm object.  We could return result but that does not have a header.
    # The fsm object has the header in fsm.header and the results in fsm._result
    return fsm

### Lets parse our Arista show ip int brief output with the *arista_eos_show_ip_interface_brief.textfsm* TextFMS template

In [10]:
ip_table = textfsm_parser("arista_eos_show_ip_interface_brief.textfsm", my_data)

Using TextFSM Template arista_eos_show_ip_interface_brief.textfsm
FSM Object fsm (basically the parsing template to use):
Value INTERFACE (\S+)
Value IP (\S+)
Value STATUS (\S+)
Value PROTOCOL (\S+)
Value MTU (\d+)

Start
  ^${INTERFACE}\s+${IP}\s+${STATUS}\s+${PROTOCOL}\s+${MTU} -> Record


FSM Object fsm.header:
['INTERFACE', 'IP', 'STATUS', 'PROTOCOL', 'MTU']

FSM Object fsm.values:
[<textfsm.parser.TextFSMValue object at 0x7fba1806fd90>, <textfsm.parser.TextFSMValue object at 0x7fba7839fd50>, <textfsm.parser.TextFSMValue object at 0x7fba1806fbd0>, <textfsm.parser.TextFSMValue object at 0x7fba1806fe10>, <textfsm.parser.TextFSMValue object at 0x7fba18080290>]

FSM Object fsm._result:
[['Ethernet1', '192.168.56.107/24', 'up', 'up', '1500'], ['Ethernet2', '192.168.194.10/24', 'up', 'up', '1500'], ['Ethernet3', '192.168.62.10/24', 'up', 'up', '1500'], ['Management1', '10.1.10.55/24', 'up', 'up', '1500']]

The contents of the result variable (same as fsm._result): 
[['Ethernet1', '192.16

In [11]:
dir(ip_table)

['GetValuesByAttrib',
 'MAX_NAME_LEN',
 'ParseText',
 'ParseTextToDicts',
 'Reset',
 '_AppendRecord',
 '_AssignVar',
 '_CheckLine',
 '_CheckRule',
 '_ClearAllRecord',
 '_ClearRecord',
 '_DEFAULT_OPTIONS',
 '_GetHeader',
 '_GetValue',
 '_Operations',
 '_Parse',
 '_ParseFSMState',
 '_ParseFSMVariables',
 '_ValidateFSM',
 '_ValidateOptions',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cur_state',
 '_cur_state_name',
 '_line_num',
 '_options_cls',
 '_result',
 'comment_regex',
 'header',
 'state_list',
 'state_name_re',
 'states',
 'value_map',
 'values']

In [12]:
help(ip_table)

Help on TextFSM in module textfsm.parser object:

class TextFSM(builtins.object)
 |  TextFSM(template, options_class=<class 'textfsm.parser.TextFSMOptions'>)
 |  
 |  Parses template and creates Finite State Machine (FSM).
 |  
 |  Attributes:
 |    states: (str), Dictionary of FSMState objects.
 |    values: (str), List of FSMVariables.
 |    value_map: (map), For substituting values for names in the expressions.
 |    header: Ordered list of values.
 |    state_list: Ordered list of valid states.
 |  
 |  Methods defined here:
 |  
 |  GetValuesByAttrib(self, attribute)
 |      Returns the list of values that have a particular attribute.
 |  
 |  ParseText(self, text, eof=True)
 |      Passes CLI output through FSM and returns list of tuples.
 |      
 |      First tuple is the header, every subsequent tuple is a row.
 |      
 |      Args:
 |        text: (str), Text to parse with embedded newlines.
 |        eof: (boolean), Set to False if we are parsing only part of the file.
 |  

In [13]:
ip_table.header

['INTERFACE', 'IP', 'STATUS', 'PROTOCOL', 'MTU']

In [14]:
ip_table._result

[['Ethernet1', '192.168.56.107/24', 'up', 'up', '1500'],
 ['Ethernet2', '192.168.194.10/24', 'up', 'up', '1500'],
 ['Ethernet3', '192.168.62.10/24', 'up', 'up', '1500'],
 ['Management1', '10.1.10.55/24', 'up', 'up', '1500']]

In [ ]:
ip_table.values

In [ ]:
table_value_list = ip_table.values

In [ ]:
len(table_value_list)

In [ ]:
for line in table_value_list:
    print(line)

---
### Lets look at the ParseTextToDicts Method

In [15]:
with open("arista_eos_show_ip_interface_brief.textfsm") as template_fh:
    # TextFSM method needs a file handle which is why we are not using our open_file function
    fsm = textfsm.TextFSM(template_fh)
    result_dict = fsm.ParseTextToDicts(my_data)

In [16]:
result_dict

[{'INTERFACE': 'Ethernet1',
  'IP': '192.168.56.107/24',
  'STATUS': 'up',
  'PROTOCOL': 'up',
  'MTU': '1500'},
 {'INTERFACE': 'Ethernet2',
  'IP': '192.168.194.10/24',
  'STATUS': 'up',
  'PROTOCOL': 'up',
  'MTU': '1500'},
 {'INTERFACE': 'Ethernet3',
  'IP': '192.168.62.10/24',
  'STATUS': 'up',
  'PROTOCOL': 'up',
  'MTU': '1500'},
 {'INTERFACE': 'Management1',
  'IP': '10.1.10.55/24',
  'STATUS': 'up',
  'PROTOCOL': 'up',
  'MTU': '1500'}]

In [17]:
result_dict[0].keys()

dict_keys(['INTERFACE', 'IP', 'STATUS', 'PROTOCOL', 'MTU'])